In [1]:
import lm_eval
import json

# 1. 运行评测，务必加上 log_samples=True
print("🚀 正在运行评测并捕获详细数据...")
results = lm_eval.simple_evaluate(
    model="hf",
    model_args="pretrained=./merged-llama3-3b-sft-v2",  # 您的模型路径
    tasks=["ceval-valid_clinical_medicine"], # 咱们只抓这一个分最低的任务来分析
    batch_size="auto",
    device="cuda:0",
    num_fewshot=5,        # 保持和之前一样的设置
    apply_chat_template=True,
    log_samples=True      # <--- 【关键参数】开启样本日志记录
)

# 2. 提取 Bad Cases
print("\n" + "="*50)
print("🕵️‍♂️ 真实 Bad Case 深度分析")
print("="*50)

# 获取样本数据
# 结构通常是: results['samples']['任务名'] -> list of dicts
task_name = "ceval-valid_clinical_medicine"
if "samples" in results and task_name in results["samples"]:
    samples = results["samples"][task_name]
    
    bad_case_count = 0
    total_count = len(samples)
    
    for i, sample in enumerate(samples):
        # sample 里的字段通常包含:
        # 'doc': 原始题目信息
        # 'target': 正确答案 (如 'A')
        # 'filtered_resps': 模型的预测结果 (通常是概率最高的那个选项)
        # 'acc': 0 表示错，1 表示对
        
        # 不同的 lm-eval 版本字段可能略有不同，这里做个兼容处理
        is_correct = sample.get('acc', 0)
        
        # 如果做错了 (acc=0)，就是一个 Bad Case
        if is_correct == 0 and bad_case_count < 5:  # 我们只打印前 5 个来看看
            bad_case_count += 1
            
            # 提取信息
            question_text = sample['doc']['question'] # 取决于 ceval 的数据结构
            correct_ans = sample['target']
            
            # 模型的预测结果。lm-eval 会算出每个选项的 logprob
            # 我们可以通过比较 logprob 找到模型选了哪个
            # 但通常 filtered_resps 或者 arguments 会包含相关信息
            # 最直接的：看 log_likelihood 最大的那个
            
            print(f"❌ Bad Case #{bad_case_count}")
            print(f"📝 题目: {question_text[:100]}...") # 只打印前100字
            print(f"✅ 正确答案: {correct_ans}")
            
            # 尝试打印模型的选择 (这里需要看具体返回结构，通常 sample 里有)
            # 如果 sample 直接给了预测结果最好，否则我们只能看到它得分是 0
            print(f"🤖 原始样本数据: {sample}") 
            print("-" * 30)

    print(f"\n📊 统计: 共扫描 {total_count} 条，发现错误 {len([s for s in samples if s.get('acc',0)==0])} 条。")

else:
    print("❌ 未在结果中找到 samples 数据，请检查 log_samples=True 是否生效或 lm-eval 版本。")

# 3. (可选) 将错题保存到文件以便后续慢慢看
with open("bad_cases.json", "w", encoding="utf-8") as f:
    json.dump(results["samples"], f, indent=4, ensure_ascii=False)
print("💾 所有详细样本已保存到 bad_cases.json")

🚀 正在运行评测并捕获详细数据...


Loading weights:   0%|          | 0/254 [00:00<?, ?it/s]

Overwriting default num_fewshot of ceval-valid_clinical_medicine from None to 5
Chat template formatting change affects loglikelihood and multiple-choice tasks. See docs/chat-template-readme.md for details.
Running loglikelihood requests:   0%|          | 0/88 [00:00<?, ?it/s]

Passed argument batch_size = auto:1. Detecting largest batch size
Determined largest batch size: 32


Running loglikelihood requests: 100%|██████████| 88/88 [00:20<00:00,  4.20it/s]



🕵️‍♂️ 真实 Bad Case 深度分析
❌ Bad Case #1
📝 题目: 附睾精液囊肿____...
✅ 正确答案: 1
🤖 原始样本数据: {'doc_id': 0, 'doc': {'id': 0, 'question': '附睾精液囊肿____', 'A': '精子自附睾管溢出至周围组织形成的囊肿', 'B': '睾丸网或输出小管扩张形成囊肿，囊肿液中可见精子或精子碎片', 'C': '囊肿为质硬的圆形包块', 'D': '囊肿与周围组织界限不清', 'answer': 'B', 'explanation': ''}, 'target': 1, 'arguments': [('<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 10 Feb 2026\n\n以下是中国关于临床医学的单项选择题，请选出其中的正确答案。<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n下列哪项不是急性心肌梗死患者溶栓治疗的禁忌证____\nA. 发病6小时，心电图ST段抬高＞0．2mY\nB. 有慢性严重高血压病史\nC. 可疑为主动脉夹层\nD. 近期(2～4周)有活动性内脏出血\n答案：<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nA<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n结核病的细胞免疫反应中起主要作用的细胞为____\nA. B淋巴细胞\nB. 巨噬细胞\nC. T淋巴细胞\nD. 嗜酸性粒细胞\n答案：<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nC<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nⅡ型呼吸衰竭的实验室指标是____\nA. $PaO_2$ 50mmHg，$PaCO_2$ 60mmHg\nB. $PaO_2$ ＞60mmHg，$PaCO_2$ ＜5